In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import HiveContext
import pandas as pd
import os
import sqlite3
import time


spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = HiveContext(sc)

con = sqlite3.connect('database.sqlite')

pddf_Player=pd.read_sql_query("SELECT * from Player",con)
schemapddf_Player = sqlContext.createDataFrame(pddf_Player)
sqlContext.registerDataFrameAsTable(schemapddf_Player, "Player")

pddf_Country=pd.read_sql_query("SELECT * from Country",con)
schemapddf_Country = sqlContext.createDataFrame(pddf_Country)
sqlContext.registerDataFrameAsTable(schemapddf_Country, "Country")

pddf_League=pd.read_sql_query("SELECT * from League",con)
schemapddf_League = sqlContext.createDataFrame(pddf_League)
sqlContext.registerDataFrameAsTable(schemapddf_League, "League")

pddf_Match=pd.read_sql_query("SELECT * from Match",con)
schemapddf_Match = sqlContext.createDataFrame(pddf_Match)
sqlContext.registerDataFrameAsTable(schemapddf_Match, "Match")

pddf_Player_Attributes=pd.read_sql_query("SELECT * from Player_Attributes",con)
schemapddf_Player_Attributes = sqlContext.createDataFrame(pddf_Player_Attributes)
sqlContext.registerDataFrameAsTable(schemapddf_Player_Attributes, "Player_Attributes")

pddf_Team=pd.read_sql_query("SELECT * from Team",con)
schemapddf_Team = sqlContext.createDataFrame(pddf_Team)
sqlContext.registerDataFrameAsTable(schemapddf_Team, "Team")

pddf_Team_Attributes=pd.read_sql_query("SELECT * from Team_Attributes",con)
schemapddf_Team_Attributes = sqlContext.createDataFrame(pddf_Team_Attributes)
sqlContext.registerDataFrameAsTable(schemapddf_Team_Attributes, "Team_Attributes")

start = time.time()

#q1
sqlContext.sql("Select player_name as Player_Name,birthday as Birthday from Player where birthday BETWEEN '1987-01-01 00:00:00' AND '1990-12-30 00:00:00' order by birthday").show()
start = time.time()

end = time.time()
print(end - start)

In [ ]:
#q2
start = time.time()
sqlContext.sql("SELECT C.name as Country,L.name as League_Name, SUM(M.home_team_goal+M.away_team_goal) as Total_Goals_Scored FROM Country C JOIN League L ON L.country_id=C.id JOIN Match M ON M.league_id=L.id group by C.name,L.name \
order by Total_Goals_Scored desc").show()
end = time.time()
print(end - start)

In [ ]:
#q3
start = time.time()
sqlContext.sql("SELECT T.team_long_name as Team_Long_Name, (avg(TA.buildUpPlaySpeed)+ ifnull(avg(TA.buildUpPlayDribbling),0) +avg(TA.buildUpPlayPassing)+ avg(TA.chanceCreationPassing)+ avg(TA.chanceCreationCrossing)+ \
avg(TA.chanceCreationShooting)+ avg(TA.defencePressure)+ avg(TA.defenceAggression)+ avg(TA.defenceTeamWidth))/9 as Average_of_Attributes from Team_Attributes TA JOIN Team T ON TA.team_api_id=T.team_api_id group by T.team_api_id \
order by Average_of_Attributes desc").show()
end = time.time()
print(end - start)

In [ ]:
#q4
start = time.time()
sqlContext.sql("SELECT T.team_long_name as Team_Name, null as  Number_of_Players, (avg(TA.buildUpPlaySpeed)+ avg(TA.buildUpPlayDribbling) +avg(TA.buildUpPlayPassing)+ avg(TA.chanceCreationPassing)+ \
avg(TA.chanceCreationCrossing)+ avg(TA.chanceCreationShooting)+ avg(TA.defencePressure)+ avg(TA.defenceAggression)+ avg(TA.defenceTeamWidth))/9 as Player_Attribute_Average from \
Team_Attributes TA JOIN Team T ON TA.team_api_id=T.team_api_id group by T.team_api_id order by Player_Attribute_Average desc").show()
end = time.time()
print(end - start)

In [ ]:
#q5
start = time.time()
sqlContext.sql("SELECT strftime('%d/%m/%Y',IQ.date) as [Date (dd/mm/yy)],IQ.season as Season,L.name as League_Name,max(goal) as Goals_scored from (SELECT date,season,league_id,sum(home_team_goal+away_team_goal) \
as goal from Match group by season,league_id,date order by season,league_id,date,goal) IQ JOIN League L ON L.id=IQ.league_id group by IQ.season,IQ.league_id").show()
end = time.time()
print(end - start)

In [ ]:
#q6
start = time.time()
sqlContext.sql("select * from (select Season, League, row_number() OVER(Partition BY League ORDER BY SUM(Goals_Scored) DESC) as [RANK] , Team_Name, sum(Goals_Scored) as Goals_Scored from \
(SELECT M.season as Season,L.name as League,T.team_long_name as Team_Name,sum(M.home_team_goal) as Goals_Scored from \
Match M JOIN \
Team T ON T.team_api_id=M.home_team_api_id JOIN League L ON L.id=M.league_id \
where M.season='2008/2009' group by T.team_long_name "
                  "UNION " 
"SELECT M.season as Season,L.name as League,T.team_long_name as Team_Name,sum(M.away_team_goal) as Goals_Scored from \
Match M JOIN \
Team T ON T.team_api_id=M.away_team_api_id JOIN League L ON L.id=M.league_id \
where M.season='2008/2009' group by T.team_long_name) group by Season, League, Team_Name ) \
where RANK<=5 order by League,RANK,Team_Name").show()
end = time.time()
print(end - start)